# The healthmate Reminder and Inquiry System 

In [ ]:
# import necessary libraries 

import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from datetime import timedelta

In [ ]:
# loading the dataset 
df = pd.read_csv("healthdata .csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
# summary statistics
df.describe()

In [ ]:
mistake=df.query("Age==""-1")

In [ ]:
mistake

In [ ]:
df.drop(index=99832,inplace=True)

In [ ]:
df.describe()

In [ ]:
# data cleaning or check for missing values 
df.isnull().sum()

In [ ]:
df.rename(columns = {"Hipertension" : "Hypertension" },inplace = True)
df.rename(columns = {"No-show" : "No_show"},inplace = True)
df.head()

In [ ]:
df["Scholarship"].value_counts()

## data visualization  

In [ ]:
plt.figure(figsize=(14,6))
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation heatmap ")


In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='Gender')

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot( data =df , x='Age',bins = 20)
plt.title("patient age distribution")

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='No_show')
plt.title("Appointment")

In [ ]:
show=df["No_show"]=="No"
noshow=df["No_show"]=="Yes"


In [ ]:
df[show].count()

In [ ]:
df[noshow].count()

In [ ]:
def attendance(df,col_name,attended,absent):
    plt.figure(figsize=[20,8])
    df[col_name][show].hist(alpha=0.5,bins=10,color="blue",label="show")
    df[col_name][noshow].hist(alpha=0.5,bins=10,color="red",label="noshow")
    plt.legend();
    plt.title("comparison acc. to Age")
    plt.xlabel("Age")
    plt.ylabel("pattents number")
attendance(df,"Age",show,noshow)

In [ ]:
def attendance(df,col_name,attended,absent):
    plt.figure(figsize=[16,5])
    df[col_name][show].value_counts(normalize=True).plot(kind="pie",label="show")
    plt.legend();
    plt.title("comparison bettween attendance by gender")

attendance(df,"Gender",show,noshow)

In [ ]:
def attendance(df,col_name,attended,absent):
    plt.figure(figsize=[16,5])
    df[col_name][noshow].value_counts(normalize=True).plot(kind="pie",label="noshow")
    plt.legend();
    plt.title("comparison bettween attendance by gender")
attendance(df,"Gender",show,noshow)

In [ ]:
plt.figure(figsize=[16,5])
df[show].groupby("Gender").Age.mean().plot(kind="bar",color="red",label="show")
df[noshow].groupby("Gender").Age.mean().plot(kind="bar",color="blue",label="noshow")
plt.legend();
plt.title("comparison acc to age ,chronic diseases")
plt.xlabel("Gender")
plt.ylabel("Mean age");

In [ ]:
plt.figure(figsize=[16,5])
df.Neighbourhood[show].value_counts().plot(kind="bar",color="blue",label="show")
df.Neighbourhood[noshow].value_counts().plot(kind="bar",color="red",label="noshow")
plt.legend();
plt.title("comparison acc. to Neighbourhood")
plt.xlabel("Neighbourhood")
plt.ylabel("pattents number");


In [ ]:
# Day of the Week Analysis
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])
df['DayOfWeek'] = df['AppointmentDay'].dt.day_name()

plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='DayOfWeek', hue='No_show')
plt.title("Appointment No-shows by Day of the Week")
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='SMS_received', hue='No_show')
plt.title("Effect of SMS Reminder on No-shows")
plt.show()

# Implementing virtual health assistant features

## Personalized Reminders

In [ ]:
# Personalized Reminders

def send_reminder(appointment_date):
    reminder_date = appointment_date - timedelta(days=1)  # Sending reminder 1 day before the appointment
    return reminder_date

df['ReminderDate'] = df['AppointmentDay'].apply(send_reminder)

# Health Tips based on Age Group and Medical Conditions
def provide_health_tips(row):
    tips = []
    
    if row['Age'] == 'Children':
        tips.append("Encourage regular physical activity and a balanced diet.")
    
    if row['Hypertension'] == 1:
        tips.append("Monitor your blood pressure regularly and follow your doctor's recommendations.")
    
    if row['Diabetes'] == 1:
        tips.append("Maintain a healthy diet and closely manage your blood sugar levels.")
    
    return '\n'.join(tips)

df['HealthTips'] = df.apply(provide_health_tips, axis=1)

# Display personalized reminders and health tips for selected patients
sample_patients = df.sample(n=5)  # Select 5 random patients for demonstration

for index, row in sample_patients.iterrows():
    print(f"Patient: {row['PatientId']}")
    print(f"Reminder Date: {row['ReminderDate']}")
    print(f"Health Tips:\n{row['HealthTips']}")
    print("="*50)

## Interactive QnA System 

We've implemented a simple function answer_patient_questions that provides responses to patient questions based on certain keywords. In a real-world scenario, you would integrate more advanced natural language processing to handle a wider range of questions.

In [ ]:
# Interactive Q&A System
def answer_patient_questions(question):
    # Placeholder logic for answering patient questions
    if "symptoms" in question.lower():
        return "If you're experiencing severe symptoms, please consult a medical professional immediately."
    elif "medication" in question.lower():
        return "Please follow your doctor's prescription and take the medication as directed."
    else:
        return "I recommend discussing this with your healthcare provider for accurate guidance."

# Simulating patient questions
sample_questions = [
    "What should I do if I experience symptoms?",
    "How should I take my medication?",
    "Can you tell me more about my medical condition?"
]

for question in sample_questions:
    answer = answer_patient_questions(question)
    print(f"Patient Question: {question}")
    print(f"Assistant Answer: {answer}")
    print("="*50)

## Insights based on Appointment history

We've created a function provide_insights that calculates and displays insights for each patient based on their appointment history. This can help patients understand their attendance patterns and adherence to appointments.

In [ ]:
# Providing Insights based on Appointment History
def provide_insights(patient_id):
    patient_df = df[df['PatientId'] == patient_id]
    total_appointments = len(patient_df)
    no_show_count = patient_df['No_show'].apply(lambda x: 1 if x == 'Yes' else 0).sum()  # Convert to 1 or 0
    show_count = total_appointments - no_show_count
    
    insights = f"Patient Insights:\n"
    insights += f"Total Appointments: {total_appointments}\n"
    insights += f"Appointments Attended: {show_count}\n"
    insights += f"Appointments Missed: {no_show_count}\n"
    insights += f"Show-up Rate: {show_count/total_appointments:.2%}\n"
    
    return insights

# Display insights for selected patients
sample_patients = df['PatientId'].sample(n=3).unique()  # Select 3 random patients for demonstration

for patient_id in sample_patients:
    insights = provide_insights(patient_id)
    print(f"Patient ID: {patient_id}")
    print(insights)
    print("="*50)